In [ ]:
# pip install pyspark
# pip install findspark 
# pip show pyspark

In [ ]:
#import findspark
#findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("lab").getOrCreate()

In [ ]:
print(f'The PySpark {spark.version} version is running...')

In [ ]:
from pyspark.sql.functions import col,column,expr,lit, instr, pow, round, bround
from pyspark.sql import Row 
from pyspark.sql.types import StructType,StructField,StringType,LongType

In [ ]:
# To Spark (in Python or R), there is no such thing as a Dataset. Everything is a DataFrame 
spark.range(2)

In [ ]:
spark.range(2).collect()

In [ ]:
df = spark.read \
        .format("json") \
        .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [ ]:
df

In [ ]:
df.schema

In [ ]:
df.printSchema()

In [ ]:
listOfRowObjects = df.collect()

In [ ]:
type(listOfRowObjects)

In [ ]:
len(listOfRowObjects)

In [ ]:
type(listOfRowObjects[0])

In [ ]:
df = spark.createDataFrame([(1, (2,2))], ["a", "b"])
df.show()
df.printSchema(1)
df.printSchema(2)

In [ ]:
df

In [ ]:
df.schema

In [ ]:
from pyspark.sql.functions import col,column,expr

In [ ]:
df.a

In [ ]:
df['a']

In [ ]:
df.select(df.a).show()

In [ ]:
df.select(df['a']).show()

In [ ]:
df.select(col('a')).show()

In [ ]:
# An expression created via the `expr` function is just a DataFrame column reference
# In the simplest case, expr("someCol") is equivalent to col("someCol")
expr('a')

In [ ]:
# All the below are same! 
print(expr('a - 5'))
print(col('a') - 5)
print(expr('a') -5)

In [ ]:
df.columns

### Records and Rows

In [ ]:
df.first()

### Creating Rows

In [ ]:
from pyspark.sql import Row 
myRow = Row("Hello", None, 1, False)

In [ ]:
myRow

In [ ]:
df.createOrReplaceTempView("dfTable")

In [ ]:
spark.sql("select * from dfTable").show(10)

### Creating DataFrames

In [ ]:
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType,LongType
myManualSchema = StructType([
    StructField('first_name', StringType(), True),
    StructField('last_name', StringType(), True),
    StructField('age', LongType(), True)
])

alice = Row("Alice", "Henderson", 25)
bob = Row("Bob", "Sanders", 28)
spark.createDataFrame([alice, bob], myManualSchema).show()

In [ ]:
# pip install sparksql-magic
# Load the extension
%load_ext sparksql_magic

In [ ]:
%%sparksql
select * from dfTable limit 250

### select and selectExpr

In [ ]:
df.select('DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME').show(2, False)

In [ ]:
from pyspark.sql.functions import expr, col, column
df.select(
    expr('DEST_COUNTRY_NAME'),
    col('DEST_COUNTRY_NAME'),
    column('DEST_COUNTRY_NAME')).show(2)

In [ ]:
df.select(col('DEST_COUNTRY_NAME'), 'DEST_COUNTRY_NAME')

In [ ]:
df.select(expr('DEST_COUNTRY_NAME as destination')).show(2)

In [ ]:
df.select(expr('DEST_COUNTRY_NAME as destination').alias('alias_destination')).show(2)

In [ ]:
df.selectExpr('DEST_COUNTRY_NAME as destination', 'DEST_COUNTRY_NAME').show(2)

In [ ]:
df.selectExpr('*', '(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withInCountry').show(10)

In [ ]:
# Aggregation over the entire DataFrame
df.selectExpr('avg(count)','count(distinct(DEST_COUNTRY_NAME))').show(2)

### Converting to Spark Types (Literals)

In [ ]:
from pyspark.sql.functions import lit
df.select('*',lit(1).alias('one')).show(2)

In [ ]:
df.select(expr('*'),lit(1).alias('one')).show(2)

### Adding Columns

In [ ]:
df.withColumn('one', lit(1)).show(2)

In [ ]:
# Another way to add a column!
df.withColumn('withInCountry', expr('DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME')).show(2)

In [ ]:
# Another way to rename a column!
df.withColumn('destination',expr('DEST_COUNTRY_NAME')).show(2)

In [ ]:
df.withColumnRenamed('DEST_COUNTRY_NAME','destination').columns

In [ ]:
dfWithLongColName = df.withColumn('Col Name With Space',expr('DEST_COUNTRY_NAME'))

In [ ]:
dfWithLongColName.show(2)

In [ ]:
dfWithLongColName.selectExpr('`Col Name With Space`', '`Col Name With Space` as `new col`').show(2)

In [ ]:
dfWithLongColName.select(col('Col Name With Space')).show(2)

In [ ]:
# We need to escape expressions that use reserved characters/keywords
dfWithLongColName.select(expr('`Col Name With Space`')).show(2)

In [ ]:
df.printSchema

In [ ]:
df.withColumn('count2', col('count').cast('int')).printSchema

In [ ]:
# df.filter(col('count') < 2).show(2)
# df.filter(expr('count') < 2).show(2)
df.filter(expr('count < 2')).show(2)

In [ ]:
df.where('count < 2').show(2)

In [ ]:
df.where(col('count') < 2).show(2)

In [ ]:
df.where(col('count') < 2)\
    .where("ORIGIN_COUNTRY_NAME != 'Croatia'")\
    .show(2)

In [ ]:
df.where(col('count') < 2).where(col('ORIGIN_COUNTRY_NAME') != 'Croatia').show(2)

In [ ]:
df.select('DEST_COUNTRY_NAME','ORIGIN_COUNTRY_NAME').distinct().count()

In [ ]:
df.select('ORIGIN_COUNTRY_NAME').distinct().count()

### Random Samples

In [ ]:
# sampling without replacement, in which a subset of the observations is selected randomly, and once an observation is selected it cannot be selected again. 
# sampling with replacement, in which a subset of observations are selected randomly, and an observation may be selected more than once.
df.sample(False,.9).count()

### Concatenating and Appending Rows (Union)

In [ ]:
from pyspark.sql import Row

lastRow = Row('New Country', 'New Country', 1)
newDF = spark.createDataFrame([lastRow], df.schema)
df.union(newDF).where('DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME').show(300)

### Sorting Rows

In [ ]:
df.sort('count').show(5)

In [ ]:
df.orderBy('count','DEST_COUNTRY_NAME').show(5)

In [ ]:
df.orderBy(col('count'), col('DEST_COUNTRY_NAME')).show(5)

In [ ]:
df.orderBy(expr('count desc')).show(5)

In [ ]:
df.orderBy(col('count').desc(), col('DEST_COUNTRY_NAME').asc()).show(5)

In [ ]:
# For optimization purposes, it's sometimes advisable to sort within each partition before
# another set of transformations
spark.read \
        .format("json") \
        .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json")\
        .sortWithinPartitions('count')

### Repartition & Coalesce
#### https://mrpowers.medium.com/managing-spark-partitions-with-coalesce-and-repartition-4050c57ad5c4

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.repartition(5)

In [ ]:
rangeDF=spark.range(10)

In [ ]:
rangeDF.rdd.getNumPartitions()

In [ ]:
rangeDF.write.csv('/Users/deepakagrawal/Desktop/data', 'overwrite')

In [ ]:
increasedPartitionedDF=rangeDF.repartition(6)

In [ ]:
increasedPartitionedDF.write.csv('/Users/deepakagrawal/Desktop/data', 'overwrite')

## Chapter 6

In [ ]:

df = spark.read.format('csv') \
          .option('header', 'true') \
          .option('inferSchema','true') \
          .load("/Applications/MAMP/htdocs/Spark-The-Definitive-Guide/data/retail-data/by-day/2010-12-01.csv")

In [ ]:
df.printSchema()

In [ ]:
df.schema.fields

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.createOrReplaceTempView('dfTable')

In [ ]:
df.show(10, False)

In [ ]:
df.select(lit(5), lit("five"), lit(5.0)).printSchema()

In [ ]:
df.select(lit(5), lit("five"), lit(5.0)).schema

In [ ]:
df.select(lit(5), lit("five"), lit(5.0)).show(10)

In [ ]:
lit('first_name')

In [ ]:
col('first_name')

In [ ]:
column('first_name')

In [ ]:
expr('first_name')

In [ ]:
def spark_to_postgres_type(spark_type):
    if spark_type == 'string':
        return 'TEXT'
    elif spark_type == 'integer':
        return 'INTEGER'
    elif spark_type == 'long':
        return 'BIGINT'
    elif spark_type == 'float':
        return 'REAL'
    elif spark_type == 'double':
        return 'DOUBLE PRECISION'
    elif spark_type == 'boolean':
        return 'BOOLEAN'
    elif spark_type == 'date':
        return 'DATE'
    elif spark_type == 'timestamp':
        return 'TIMESTAMP'
    else:
        return 'TEXT'

table_name = 'my_table'
create_table_sql = f"CREATE TABLE {table_name} ("
for field in df.schema.fields:
    field_name = field.name
    field_type = field.dataType.typeName()
    postgres_field_type = spark_to_postgres_type(field_type)
    create_table_sql += f"{field_name} {postgres_field_type}, "

create_table_sql = create_table_sql[:-2] # Remove the last comma and the space
create_table_sql += ')'
print(create_table_sql)

In [ ]:
df.where(col('InvoiceNo') != 536365) \
  .select('InvoiceNo','Description') \
  .show(5, False)

In [ ]:
df.where('InvoiceNo = 536365').show(5, False)

In [ ]:
df.where('InvoiceNo <> 536365').show(5, False)

In [ ]:
priceFilter = col('UnitPrice') > 600
descripFilter = instr(df.Description, 'POSTAGE') >= 1
df.where(df.StockCode.isin('DOT')).where(priceFilter | descripFilter).show()

In [ ]:
DOTCodeFilter = col('StockCode') == 'DOT'
priceFilter = col('UnitPrice') > 600
descripFilter = instr(df.Description, 'POSTAGE') >= 1
df.withColumn('isExpensive', DOTCodeFilter & (priceFilter | descripFilter)) \
  .where('isExpensive') \
  .select('UnitPrice', 'StockCode', 'Description',  'isExpensive').show()

In [ ]:
# Express Filters as SQL statements 
df.withColumn('isExpensive', expr('NOT UnitPrice <= 250')) \
  .where('isExpensive') \
  .select('Description','UnitPrice').show()

In [ ]:
# Working with null data when creating Boolean expressions
df.where(col('Description').eqNullSafe("hello")).show()

In [ ]:
fabricatedQuantity = pow(col('Quantity') * col('UnitPrice'), 2) + 5
df.select(expr('CustomerID'), fabricatedQuantity.alias('realQuantity')).show()

In [ ]:
# As SQL expression
df.selectExpr('CustomerID', '(POWER((Quantity * UnitPrice), 2) + 5) as realQuantity').show()

In [ ]:
# Rounding
df.select(round(lit('2.5')), bround(lit('2.5'))).show(1)

In [ ]:
# Compute Summary Statistics 
df.describe().show()

In [ ]:
# Add a unique ID to each row
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(5)

### Working With Strings

In [ ]:
from pyspark.sql.functions import initcap
df.select(col('Description')).show(10, False)
df.select(initcap(col('Description'))).show(10, False)

In [ ]:
from pyspark.sql.functions import lower, upper
df.select(col('Description'), \
          lower(col('Description')), \
          upper(lower(col('Description')))) \
  .show(5, False)

In [ ]:
from pyspark.sql.functions import lit, ltrim, rtrim, trim, lpad, rpad
df.select(
    ltrim(lit('     HELLO      ')).alias('ltrim'),
    rtrim(lit('     HELLO      ')).alias('rtrim'),
    trim(lit('     HELLO      ')).alias('trim'),
    lpad(lit('HELLO'), 3, '$').alias('lpad'),
    rpad(lit('HELLO'), 10, '#').alias('rpad')
  ).show(1)

### Regular Expressions

In [ ]:
from pyspark.sql.functions import regexp_replace

regex_string = 'BLACK|WHITE|RED|GREEN|BLUE'
df.select(
    col('Description'),
    regexp_replace(
        col('Description'), regex_string, '{COLOR}').alias('color_clean')
    ).show(10, False)

In [ ]:
from pyspark.sql.functions import translate
df.select(
    col('Description'),
    translate(
        col('Description'), 'LET', '137')
    ).show(10, False)

In [ ]:
# Pulling out the first mentioned color:
from pyspark.sql.functions import regexp_extract
extract_string = 'BLACK|WHITE|RED|GREEN|BLUE'
df.select(
    col('Description'),
    regexp_extract(
        col('Description'), extract_string, 0).alias('color_clean')
    ).show(10, False)

In [ ]:
from pyspark.sql.functions import instr 
containsBlack = instr(col('Description'), 'BLACK') >=1
containsWhite = instr(col('Description'), 'WHITE') >=1
df.withColumn('hasSimpleColor', containsBlack | containsWhite) \
  .where('hasSimpleColor') \
  .select('Description') \
  .show(3, False)

In [ ]:
from pyspark.sql.functions import expr, locate
simpleColors = ['black', 'white', 'red', 'green', 'blue']
def color_locator(column, color_string):
    return locate(color_string.upper(), column).cast('boolean').alias('is_' + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr('*'))
df.select(*selectedColumns).where(expr('is_white AND is_red')).select('Description').show(3, False)

### Working with Dates and Timestamps

In [ ]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(1).withColumn('today', current_date()).withColumn('now', current_timestamp())
dateDF.show(1, False)
dateDF.printSchema()

In [ ]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col('today'), 5), date_add(col('today'), 5)).show(1, False)

In [ ]:
# The name datediff is a shorthand derived from the words "date" and "difference"
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn('week_ago', date_sub(col('today'), 7)) \
      .select(datediff(col('week_ago'), col('today'))).show(1)

In [ ]:
dateDF.select(
    to_date(lit('2016-01-01')).alias('start'), 
    to_date(lit('2017-01-01')).alias('end')) \
    .select(months_between(col('start'), col('end'))).show(1)

In [ ]:
spark.range(1).withColumn('date', lit('2017-01-01')).printSchema()

In [ ]:
spark.range(1).withColumn('date', lit('2017-01-01')) \
     .select(to_date(col('date'))).printSchema()

In [ ]:
# Spark will not throw an error if it can't parse the date; rather, it will just return null
dateDF.select(to_date(lit('2016-20-12')), to_date(lit('2016-12-20'))).show(1)

In [ ]:
dateFormat = 'yyyy-dd-MM'
dateDF.select(to_date(lit('2016-20-12'), dateFormat), to_date(lit('2016-12-20'), dateFormat)).show(1)

In [ ]:
dateFormat = 'yyyy-MM-dd'
dateDF.select(to_date(lit('2016-20-12'), dateFormat), to_date(lit('2016-12-20'), dateFormat)).show(1)

In [ ]:
from pyspark.sql.functions import to_timestamp
dateFormat = 'yyyy-dd-MM'
dateDF.select(to_date(lit('2016-20-12'), dateFormat).alias('date')) \
      .select(to_timestamp(col('date'))).show()